Primme Performance Comparison 
=============================
All tests run on a Ryzen 2700X with 32GB of RAM available

In [3]:
#!pip install primme sklearn
#!pip install -e git+https://github.com/bwlewis/irlbpy.git#egg=irlbpy
#!pip install -e git+https://github.com/jakevdp/pypropack.git#egg=pypropack
#!pip install tabulate

Defaulting to user installation because normal site-packages is not writeable
  Using cached tabulate-0.8.9-py3-none-any.whl (25 kB)


In [5]:
import primme, irlb, timeit, sklearn.decomposition, scipy.sparse, numpy as np
from tabulate import tabulate
#from pypropack import svdp



PRIMME eigsh is based on Davidson-type methods and they may be faster than Lanczos/Arnoldi based method in difficult problems that eigenpairs take many iterations to converge or an efficient preconditioner is available.l

In [6]:
#------------------------
#EigenValue Tests
#------------------------

normA = 12000
headers = ["test", "time", "matvec", "rnorm"]

table = list()
tolerance = 1e-5
A = np.diag(np.asarray(range(1,12001), dtype=np.float32), 0)
for i in range(1,11999):
  A[i,i+1] = 1
  A[i+1,i] = 1

#PRIMME performance test
start = timeit.default_timer()
evals, evecs, stats = primme.eigsh(A, 2, tol=tolerance, return_stats =True, return_history=True)
end = timeit.default_timer()
table.append(["Primme", end - start, stats["numMatvecs"], max(stats['rnorms'])])

#scipy.sparse.linalg.lobpcg performance test
y = np.eye(12000, 2)
rng = np.random.default_rng()
X = rng.random((12000, 2))
start = timeit.default_timer()
evals, evecs = scipy.sparse.linalg.lobpcg(A, X=X, tol=tolerance*normA, largest=True, maxiter=1000)
end = timeit.default_timer()
rnorm = np.linalg.norm(np.matmul(A, evecs) - (evecs.dot(np.diag(evals))), axis=0)
table.append(["sp.sparse.linalg.lobpcg", end - start, "---", max(rnorm)])
print(tabulate(table, headers=headers, tablefmt="rst"), "\n")


=======================  =======  ========  ========
test                        time  matvec       rnorm
=======================  =======  ========  ========
Primme                    36.922  568       0.120693
sp.sparse.linalg.lobpcg  126.848  ---       0.115554
=======================  =======  ========  ======== 



By default PRIMME tries to guess the best configuration, but a little hint can help sometimes. Convergence can be accelerated by providing PRIMME with a preconditioner 

In [4]:
#------------------------------------------
#EigenValue Preconditioner comparison Tests
#------------------------------------------

table = list()
#PRIMME performance test
start = timeit.default_timer()
evals, evecs, stats = primme.eigsh(A, 2, which= 'SM', tol=tolerance, return_stats =True, return_history=True)
end = timeit.default_timer()
table.append(["Primme", end - start, stats["numMatvecs"], max(stats['rnorms'])])


diagInv = np.diag(np.reciprocal(np.asarray(range(1,12001), dtype=np.float32)), 0)
#PRIMME Preconditioned performance test
start = timeit.default_timer()
evals, evecs, stats = primme.eigsh(A, 2, OPinv= diagInv, tol=tolerance, return_stats =True, which= 'SM', return_history=True)
end = timeit.default_timer()
table.append(["Primme Prec", end - start, stats["numMatvecs"], max(stats['rnorms'])])
print(tabulate(table, headers=headers, tablefmt= "rst"), "\n")


===========  ========  ========  ========
test             time    matvec     rnorm
===========  ========  ========  ========
Primme       36.6693        458  0.118197
Primme Prec   1.94905        29  0.108924
===========  ========  ========  ======== 



For svds problems the package has a similar intereface. PRIMME svds may perform as good as similar methods in the packages scipy.linalg.sparse and irlb in solving few singular values. PRIMME can take advantage of a light matrix-vector product. For larger problems PRIMME performs similarly.

In [8]:
#------------------------
#Singular Value Tests
#------------------------

A = np.random.normal(size=(6000,6000))
normA = 154.8
k = [2,100]

for n in k:
  table = list()
  #PRIMME performance test
  #Because A is a dense matrix we use a large maxBlockSize to take advantage of BLAS3.
  #For sparse matirices much smaller maxBlockSize. Experiment based on you problem.
  start = timeit.default_timer()
  u, sdvals, vt, stats = primme.svds(A, n, tol=tolerance, maxBlockSize=n, return_stats=True)
  end = timeit.default_timer()
  table.append(["Primme", end - start, stats["numMatvecs"], max(stats['rnorms'])])
  
  #scipy.sparse.linalg.svds performance test
  start = timeit.default_timer()
  u, sdvals, vt = scipy.sparse.linalg.svds(A, n, tol=tolerance*normA, return_singular_vectors=True)
  end = timeit.default_timer()
  rnorm = np.linalg.norm(A.dot(vt.T.conj()) - u.dot(np.diag(sdvals)),axis=0)**2+np.linalg.norm( u.T.conj().dot(A).T.conj() -(vt.T.conj()).dot(np.diag(sdvals)), axis=0)**2
  table.append(["sp.sparse.linalg.svds", end - start, "---", max(rnorm)])
  
  #irlb performance test
  start = timeit.default_timer()
  u, sdvals, v, it, mprod  = irlb.irlb(A, n, tol=tolerance, maxit = 1000)
  end = timeit.default_timer()
  rnorm = np.linalg.norm(A.dot(v) - u.dot(np.diag(sdvals)),axis=0)**2+np.linalg.norm( u.T.conj().dot(A).T.conj() -v.dot(np.diag(sdvals)), axis=0)**2
  table.append(["irlb", end - start, "---", max(np.sqrt(rnorm))])
  
  if n == 2:
    #sklearn.decomposition permormance test
    start = timeit.default_timer()
    svdals = sklearn.decomposition.TruncatedSVD(n_components= n, tol=tolerance, n_iter = 1000)
    svdals.fit(A)
    end = timeit.default_timer()
    table.append(["sklearn", end - start, '---', '---'])
  print(tabulate(table, headers=headers, tablefmt="rst"), "\n")

=====================  =========  ========  ======================
test                        time  matvec    rnorm
=====================  =========  ========  ======================
Primme                  10.3734   546       0.0007492027131764414
sp.sparse.linalg.svds    5.81644  ---       4.2308075210909703e-10
irlb                     5.35749  ---       0.0011904085013419677
sklearn                154.079    ---       ---
=====================  =========  ========  ====================== 

=====================  =======  ========  ===========
test                      time  matvec          rnorm
=====================  =======  ========  ===========
Primme                 29.0386  6052      0.00212599
sp.sparse.linalg.svds  34.1852  ---       1.39758e-13
irlb                   39.979   ---       0.00154099
=====================  =======  ========  =========== 

